### Import Dependencies ###


In [1]:
import openai
import instructor
from qdrant_client import QdrantClient 
from pydantic import BaseModel, Field

In [2]:
from dotenv import load_dotenv
import os

load_dotenv("../../.env")

True

In [3]:
prompt = """
    You are a helpful assistant. 
    Return an answer to the question. 
    Question: What is your name
"""

In [4]:
response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0
)
print (response.choices[0].message.content)

I am an AI language model created by OpenAI. You can call me ChatGPT. How can I assist you today?


In [5]:
client = instructor.from_openai(openai.OpenAI())

In [6]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer of the question.")
    reasoning: str = Field(description="The reasoning of the answer.")

In [7]:
response,raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": prompt}
    ],
    temperature=0,
    response_model=RAGGenerationResponse
)

In [8]:
response

RAGGenerationResponse(answer='My name is ChatGPT.', reasoning='I am an AI language model created by OpenAI, and I am commonly known as ChatGPT.')

In [9]:
raw_response

ChatCompletion(id='chatcmpl-D0tw5Dz647vxfPozu3DHBZSTFLsFD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_M3IfgX2KwJYdsW91NLrQbR9x', function=Function(arguments='{"answer":"My name is ChatGPT.","reasoning":"I am an AI language model created by OpenAI, and I am commonly known as ChatGPT."}', name='RAGGenerationResponse'), type='function')]))], created=1769107389, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=33, prompt_tokens=115, total_tokens=148, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### RAG Example

In [10]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer of the question.")

In [19]:

from openai.types.shared import responses_model


def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    
    return response.data[0].embedding

def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):
    
    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "datamodel":answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [15]:
import qdrant_client


qdrant_client = QdrantClient(url="http://localhost:6333")

In [20]:
output = rag_pipeline("Can I get a charging calbe? Please suggest a good one.", qdrant_client)

In [21]:
output

{'datamodel': RAGGenerationResponse(answer='Based on the available products, here are some good charging cable options for you:\n\n1. GREPHONE 2 Pack USB C to Lightning Cable, 6 FT, MFi Certified - This cable supports fast charging (PD Fast Charge 3A/30W max) and is compatible with a wide range of iPhone and iPad models. It has a durable design with reinforced material and aluminum joints. Comes in a pack of 2.\n\n2. iPhone Charger Cord Lightning Cables, Original 2022 Upgraded [3Pack 3ft] - These are Apple MFi Certified original lightning cables, known for durability and fast charging (3.0A). They come in a pack of 3 and are compatible with many iPhone and iPad models.\n\n3. 5 in 1 USB C to Multi Charging Cable 3M/10Ft - This is a versatile multi-charging cable with connectors for Lightning, USB C, and Micro USB, allowing you to charge multiple devices simultaneously. It is Apple MFi Certified and 10 feet long.\n\nIf you want a simple, reliable iPhone charging cable, the Original 2022 